# <font color="#003660">Applied Machine Learning for Text Analysis (M.184.5331)</font>


# <font color="#003660">Session 9: LLM-based Apps with LangChain</font>

# <font color="#003660">LLM Agents</font>

<center><br><img width=256 src="https://raw.githubusercontent.com/olivermueller/aml4ta-2021/main/resources/dag.png"/><br></center>

<p>

<div>
    <font color="#085986"><b>By the end of this lesson, you ...</b><br><br>
        ... will know how to implement structured outputs in LLMs. <br>
        ... will know how apply this to solve a real-world task in LangChain.
    </font>
</div>
</p>

The following content is heavily inspired by the following excellent sources:

* [LangChain Academy](https://academy.langchain.com/)
* [Introduction to LangChain Agents](https://github.com/langchain-ai/langchain-academy/blob/main/module-1/agent.ipynb)
* [LangChain Docs (Python)](https://python.langchain.com/)

In [ ]:
!pip install -U langchain langchain-community langchain-ollama langgraph colab-xterm

# Agent

We want to build an Agent that consists a router (graph).

* The chat model will decide to make a tool call or not based upon the user input
* A conditional edge routes to a node that calls our tool or simply ends the route with an end node
* If multiple tool calls are necessary as in a mathematical environment where we want to apply multiple additions, subtractions, multiplications or divisions, the model needs a route to jump back and call the tools again.

We can solve this by simply pass that `ToolMessage` *back to the model*?

We can let it either (1) call another tool or (2) respond directly.

This is the intuition behind [*Reason+Act (ReAct)*](https://doi.org/10.48550/arXiv.2210.03629), a general agent architecture.
  
* `reason` - let the model reason about the input (e.g., call a tool or just respond directly)
* `act` - let the model call specific tools
* `observe` - pass the tool output back to the model

* (`answer` - answer the question after iterations of reason-act-observe)


This [general purpose architecture](https://blog.langchain.dev/planning-for-agents/) can be applied to many types of tools.

![Screenshot 2024-08-21 at 12.45.43 PM.png](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66dbac0b4a2c1e5e02f3e78b_agent2.png)

[Source: LangChain Academy GitHub](https://github.com/langchain-ai/langchain-academy/blob/main/module-1/agent.ipynb)

In [ ]:
%load_ext colabxterm

In [ ]:
%%capture --no-stderr

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

In [ ]:
%xterm # Copy this command in the Xterm starting below: HOST=0.0.0.0 ollama serve

In [ ]:
!ollama pull qwen2.5:1.5b
!ollama pull qwen2.5:7b

In [ ]:
from langchain_ollama import ChatOllama
from langchain_core.tools import tool

from langgraph.graph import MessagesState
from langchain_core.messages import HumanMessage, SystemMessage

from langgraph.graph import START, StateGraph
from langgraph.prebuilt import tools_condition
from langgraph.prebuilt import ToolNode
from IPython.display import Image, display

In [ ]:
@tool
def multiply(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b

@tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b

@tool
def divide(a: int, b: int) -> float:
    """Divide a and b.

    Args:
        a: first int
        b: second int
    """
    return a / b

@tool
def subtract(a: int, b: int) -> int:
    """Subtract b from a.

    Args:
        a: first int
        b: second int
    """
    return a - b

tools = [add, subtract, multiply, divide]

LLM_NAME = "qwen2.5:7b"

llm = ChatOllama(
    model=LLM_NAME,
    temperature=0,
    seed=42
)

llm_with_tools = llm.bind_tools(tools)

Let's create our LLM and prompt it with the overall desired agent behavior.

In [ ]:
# System message
sys_msg = SystemMessage(content="You are a helpful assistant tasked with performing arithmetic on a set of inputs.")

We will use a `MessagesState` and define a `Tools` node with our list of tools.

The `Assistant` node is just our model with bound tools.

In [ ]:
# Node
def assistant(state: MessagesState):
   return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]}



We create a graph with `Assistant` and `Tools` nodes.

We add `tools_condition` edge, which routes to `End` or to `Tools` based on  whether the `Assistant` calls a tool.

In [ ]:
# Graph
builder = StateGraph(MessagesState)

# Define nodes: these do the work
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

# Define edges: these determine how the control flow moves
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    # If the latest message (result) from assistant is a tool call -> tools_condition routes to tools
    # If the latest message (result) from assistant is a not a tool call -> tools_condition routes to END
    tools_condition,
)
react_graph = builder.compile()

# Show
display(Image(react_graph.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
# Graph
builder = StateGraph(MessagesState)

# Define nodes: these do the work
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

# Define edges: these determine how the control flow moves
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    # If the latest message (result) from assistant is a tool call -> tools_condition routes to tools
    # If the latest message (result) from assistant is a not a tool call -> tools_condition routes to END
    tools_condition,
)

Now, we add one new step:

We connect the `Tools` node *back* to the `Assistant`, forming a loop.

* After the `assistant` node executes, `tools_condition` checks if the model's output is a tool call.
* If it is a tool call, the flow is directed to the `tools` node.
* The `tools` node connects back to `assistant`.
* This loop continues as long as the model decides to call tools.
* If the model response is not a tool call, the flow is directed to END, terminating the process.

In [ ]:
builder.add_edge("tools", "assistant")
react_graph = builder.compile()

# Show
display(Image(react_graph.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
messages = [HumanMessage(content="Add 3 and 4. Multiply the output by 2. Divide the output by 10")]
messages = react_graph.invoke({"messages": messages})

In [ ]:
for m in messages['messages']:
    m.pretty_print()

# Your Task

## Implement Admin Verification Tool and Agent

You are tasked with creating a system to verify if a user is an admin based on their provided username and passphrase. Your system should utilize a **tool** and an **agent** to process and respond to user requests.

#### Requirements:

1. **Verification Tool**:
   - Create a function named `verify_admin` that takes a username and passphrase as arguments and returns `True` if both are correct (`username="admin"` and `passphrase="adminpassword"`). Otherwise, return `False`.

2. **Agent**:
   - Implement an agent to interact with users. The agent must use the `verify_admin` tool to verify the admin credentials.
   - The agent should respond with an appropriate message based on the verification result:
     - If the credentials are correct, respond with: `"Verification successful! Welcome, admin."`
     - If the credentials are incorrect, respond with: `"Verification failed. Invalid credentials."`

3. **Messages Completion**:
   - Complete the final messages at the end of the code to ensure that the agent can respond to two user requests:
     - User 1: `"Please verify me: I am the admin and my passphrase is adminpassword"`
     - User 2: `"Please verify me: I am the admin and my passphrase is admin-assword"`


**Hint:** It is mostly copy and paste.

In [ ]:
LLM_NAME = "qwen2.5:7b"

llm = ChatOllama(
    model=LLM_NAME,
    temperature=0,
    seed=42
)

In [ ]:
@tool
def verify_admin(username: str, passphrase: str) -> bool:
    # ToDo: implement tool
    pass

tools = [verify_admin, ]
llm_with_tools = llm.bind_tools(tools)

In [ ]:
# ToDo: implement system message


In [ ]:
# ToDo: implement assistant node


In [ ]:
# ToDo: implement graph


In [ ]:
messages = [HumanMessage(content="Please verify me: I am the admin and my passphrase is adminpassword")]
messages = react_graph.invoke({"messages": messages})

for m in messages['messages']:
    m.pretty_print()

In [ ]:
messages = [HumanMessage(content="Please verify me: I am the admin and my passphrase is admin-assword")]
messages = react_graph.invoke({"messages": messages})

for m in messages['messages']:
    m.pretty_print()